In [504]:
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import mysql.connector
import sqlite3
from datetime import datetime, date

In [186]:
# Get the npi list
with open('NPI.txt') as file:  
    data = file.read().split(',') 

In [285]:
# Initiate web driver
driver = webdriver.Chrome(executable_path='./bin/chromedriver')

In [507]:
# Initiate database. If the data already exists in database, resume crawling. 
def create_database(database_name, data):
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()
    try:
        statement = '''
            CREATE TABLE 'CMS_national_provider'(
                'id' INTEGER PRIMARY KEY,
                'npi' INTEGER,
                'date' TIMESTAMP,
                'abimid' INTEGER,
                'specialty' TEXT,
                'status' TEXT,
                'spec2' TEXT,
                'specdate' DATE
            ) 
        '''
        cursor.execute(statement)
        print('Successfully created database, start crawling...')
        conn.close()
        return data
    except:
        # SELECT last record 
        statement = '''
            SELECT * FROM 'CMS_national_provider'
            WHERE id = (SELECT MAX(id)  FROM 'CMS_national_provider')
        ''' 
        cursor.execute(statement)
        conn.close()
        
        # SELECT crawled data record 
        conn = sqlite3.connect(database_name)
        cursor = conn.cursor()
        statement = '''
            SELECT npi FROM 'CMS_national_provider'
        ''' 
        cursor.execute(statement)
        crawled_npi = cursor.fetchall()
        conn.close()
        if len(crawled_npi) == 0: #
            print('There is no record in data set, start crawling...')
            return data 
        else:
            existing_npi = [str(e[0]) for i,e in enumerate(crawled_npi)]
            print('There are %d existing records...resume crawling...' % (len(crawled_npi)))
            tocrawl_npi = [i for i in data if i not in existing_npi] 
            return tocrawl_npi

In [ ]:
# in main function
for i,npi in enumerate(data[:2]):
        if data.index(npi) == -1:
            driver.close()
            break

In [286]:
def crawl_data(npi): 
    
    driver.get('https://www.abim.org/verify-physician')
    search_box = driver.find_element_by_name('npi')
    search_box.send_keys(npi)
    search_button = driver.find_elements_by_name('type')[2]
    search_button.click()
    time.sleep(2)
    try:
        html = driver.find_element_by_class_name("abim_voc-profile")
    except:
        print("%s doesnt exist/IP address is banned by %s" %s (npi,'https://www.abim.org/verify-physician'))
    return html  

In [532]:
# take a crawled html from webdriver's results
def parse(html):
    elements = html.text.split('\n')
    date_index = elements.index("TODAY'S DATE")
    abimID_index = elements.index("ABIM ID")
    status_index = elements.index("CURRENT CERTIFICATION STATUS:")
    initCert_index = elements.index("INITIAL CERTIFICATIONS:")
    date = elements[date_index+1]
    abim_id = elements[abimID_index+1]
    status = elements[status_index+1:initCert_index-1]
    initial_certifications = elements[initCert_index+1:]
    return date, abim_id, status, initial_certifications

# when there is multiple certifiactions and the order of status/certification are not inlined
def save_into_database(npi,date,abim_id,status,initial_certifications):
    for s in status:
        time_stamp = datetime.strptime(date,'%b %d, %Y').strftime('%Y-%m-%d')
        certification_row = [npi,time_stamp,abim_id]
        ss = s.split(': ')
        certification_row += ss
        discipline = ss[0]
        current_certified_status = ss[1]
        for i in initial_certifications:
            ii = i.split(': ')
            i_discipline = ii[0]
            certified_time = ii[1]
            if discipline == i_discipline:
                certification_row.append(i_discipline)
                certification_row.append(certified_time)
                certification_row.insert(0,None)
                
#         print(certification_row)
        insert_data(certification_row)

In [511]:
# insert values into table 
def insert_data(row):
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()
    statement = "INSERT INTO 'CMS_national_provider' "
    statement += 'VALUES (?,?,?,?,?,?,?,?)'
    cursor.execute(statement,row)
    conn.commit()
    conn.close

In [536]:
def main(database_name, npi_list):
    data = create_database(database_name,npi_list)
    if len(data) == 0:
        print("All data is crawled and saved in %s" % (database_name,))
        fetch_dataset(database_name)
        return 'Crawling Finished'
   
    for npi in data:
        html = crawl_data(npi)
        d,a,s,i = parse(html)
        save_into_database(npi,d,a,s,i)
    print('Crawling Finished')
    
    fetch_dataset(database_name)

In [534]:
# fetch the data set
def fetch_dataset(database_name):
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()
    statement = '''
        SELECT * FROM 'CMS_national_provider'
    '''
    cursor.execute(statement)
    for i in cursor.fetchall():
        print(i)
    conn.close()

In [537]:
# main function
main('test.db',data)

There are 12 existing records...resume crawling...
All data is crawled and saved in test.db
(1, 1417280017, '2020-02-26', 332931, 'Internal Medicine', 'Certified', 'Internal Medicine', 2012)
(2, 1427044981, '2020-02-26', 113026, 'Endocrinology, Diabetes and Metabolism', 'Certified', 'Endocrinology, Diabetes and Metabolism', 1989)
(3, 1457616120, '2020-02-26', 364882, 'Endocrinology, Diabetes and Metabolism', 'Certified', 'Endocrinology, Diabetes and Metabolism', 2017)
(4, 1457616120, '2020-02-26', 364882, 'Internal Medicine', 'Certified', 'Internal Medicine', 2015)
(5, 1487620415, '2020-02-26', 181792, 'Focused Practice in Hospital Medicine', 'Certified', 'Focused Practice in Hospital Medicine', 2018)
(6, 1508819434, '2020-02-26', 227645, 'Cardiovascular Disease', 'Certified', 'Cardiovascular Disease', 2012)
(7, 1508819434, '2020-02-26', 227645, 'Interventional Cardiology', 'Certified', 'Interventional Cardiology', 2015)
(8, 1538153176, '2020-02-26', 181063, 'Internal Medicine', 'Certi

'Crawling Finished'